In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)


2025/12/08 13:00:58 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/08 13:00:58 INFO mlflow.store.db.utils: Updating database tables
2025/12/08 13:00:58 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/08 13:00:58 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2025/12/08 13:00:58 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/08 13:00:58 INFO alembic.runtime.migration: Will assume non-transactional DDL.


In [5]:
for elem in client.search_experiments():
    print(elem)

<Experiment: artifact_location='/workspaces/mlops-project/03-tracking/mlruns/1', creation_time=1764778521062, experiment_id='1', last_update_time=1765012433941, lifecycle_stage='active', name='my_new_exp', tags={'mlflow.experimentKind': 'custom_model_development'}>


In [6]:
client.create_experiment(name="second-exp")

'2'

In [16]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=['1'],
    filter_string="metrics.rmse > 5.5",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [17]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:4f}")

run id: 59830a8ce0f94c89b78ee414222b32db, rmse: 5.514061
run id: f7c023bd281b4984b13d7043666a29ee, rmse: 5.892172
run id: be653fe9940246bfb5b7284d6a98b827, rmse: 5.912991
run id: 44959528af2e4f6c970d1073f5874d47, rmse: 6.037890
run id: e000d2c9b64f47e18fd6e40f18a49eec, rmse: 6.046362


In [18]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [21]:
model_uri = "runs:/3fa2551032254fc3a34858da1e3803ac/mlflow"
mlflow.register_model(model_uri=model_uri, name='nyc-taxi')

Registered model 'nyc-taxi' already exists. Creating a new version of this model...
2025/12/08 13:19:35 WARNING mlflow.tracking._model_registry.fluent: Run with id 3fa2551032254fc3a34858da1e3803ac has no artifacts at artifact path 'mlflow', registering model based on models:/m-9af65b3e81374ba291bacb1f5fce7da9 instead
Created version '3' of model 'nyc-taxi'.


<ModelVersion: aliases=[], creation_timestamp=1765196375352, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1765196375352, metrics=None, model_id=None, name='nyc-taxi', params=None, run_id='3fa2551032254fc3a34858da1e3803ac', run_link=None, source='models:/m-9af65b3e81374ba291bacb1f5fce7da9', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [22]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1765194486506, deployment_job_id=None, deployment_job_state=None, description='This is XGBoost predictor.', last_updated_timestamp=1765196375352, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1765196375352, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1765196375352, metrics=None, model_id=None, name='nyc-taxi', params=None, run_id='3fa2551032254fc3a34858da1e3803ac', run_link=None, source='models:/m-9af65b3e81374ba291bacb1f5fce7da9', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi', tags={}>]

In [24]:
latest_versions = client.get_latest_versions(name='nyc-taxi')

C:\Users\trinh\AppData\Local\Temp\ipykernel_8512\4269581515.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name='nyc-taxi')


In [27]:
latest_versions[0].current_stage

'None'

In [28]:
client.transition_model_version_stage(
    name = 'nyc-taxi',
    version=2,
    stage='Staging'
)

C:\Users\trinh\AppData\Local\Temp\ipykernel_8512\1887588557.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1765194502595, current_stage='Staging', deployment_job_state=None, description='', last_updated_timestamp=1765198868294, metrics=None, model_id=None, name='nyc-taxi', params=None, run_id='a84393f95fe2483d9b34b6aaa72f3189', run_link='', source='models:/m-971544a5a7f847c79664956380f8a1fa', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=2>